In [1]:
import pandas as pd
from pathlib import Path

In [2]:
PROJECT_ROOT = !git rev-parse --show-toplevel
PROJECT_ROOT = PROJECT_ROOT[0]
PROJECT_ROOT = Path(PROJECT_ROOT)

In [3]:
df = pd.read_csv(str(PROJECT_ROOT / 'data' / "interim" / "data.csv"))

In [7]:
df.Supermarket.drop_duplicates()

0    woolworths
Name: Supermarket, dtype: object

In [121]:
grouped = df.groupby(["category", "unit_quantity"]).count().iloc[:, 0].to_frame("Count")
grouped.reset_index(inplace=True)
   # Get index of the original for which `Count` is higher
idx = grouped.groupby(["category"])['Count'].transform(max) == grouped['Count']
grouped[idx]

,category,unit_quantity,Count
0,banana,1EA,5
1,eggs,100G,22
5,full cream milk,1L,38


In [128]:
mask = (df["category"].isin(grouped.loc[idx, "category"].values)) & (df["unit_quantity"].isin(grouped.loc[idx, "unit_quantity"].values))
subset = df[mask]

In [129]:
subset

,availability,brand,category,datetime,name,pic,product_price,product_quantity,unit_price,unit_quantity,supermarket
0,True,NaN,full cream milk,2021-07-07 22:26:03.791277,Woolworths Drought Relief Full Cream Milk 2l,NaN,2.39,NaN,1.20,1L,woolworths
1,True,NaN,full cream milk,2021-07-07 22:26:03.792768,Dairy Farmers Permeate Free Full Cream Milk 3l,NaN,4.90,NaN,1.63,1L,woolworths
2,True,NaN,full cream milk,2021-07-07 22:26:03.794052,Woolworths Drought Relief Full Cream Milk 3l,NaN,3.59,NaN,1.20,1L,woolworths
3,True,NaN,full cream milk,2021-07-07 22:26:03.795327,Woolworths Full Cream Milk 1l,NaN,1.29,NaN,1.29,1L,woolworths
4,True,NaN,full cream milk,2021-07-07 22:26:03.796657,Farmers Own Full Cream Milk 3l,NaN,4.50,NaN,1.50,1L,woolworths
...,...,...,...,...,...,...,...,...,...,...,...
67,True,NaN,banana,2021-07-07 22:27:52.624396,Cavendish Bananas each,NaN,0.81,NaN,0.81,1EA,woolworths
68,True,NaN,banana,2021-07-07 22:27:52.626208,Macro Organic Banana each,NaN,0.99,NaN,0.99,1EA,woolworths
69,True,NaN,banana,2021-07-07 22:27:52.627517,Bananas Kids 5 pack,NaN,2.50,NaN,2.50,1EA,woolworths
70,True,NaN,banana,2021-07-07 22:27:52.628798,Eat Later Cavendish Bananas each,NaN,0.81,NaN,0.81,1EA,woolworths


In [137]:
grouped = subset.groupby(["category", "supermarket"])["unit_price"].mean().to_frame(name="Average Unit Price")
grouped["Median Unit Price"] = subset.groupby(["category", "supermarket"])["unit_price"].median()
grouped.reset_index(inplace=True)
grouped.merge(subset[["category", "supermarket", "unit_quantity"]], how="left", on=["category", "supermarket"]).drop_duplicates()

,category,supermarket,Average Unit Price,Median Unit Price,unit_quantity
0,banana,woolworths,1.220000,0.990,1EA
5,eggs,woolworths,0.848571,0.860,100G
27,full cream milk,woolworths,2.061190,1.875,1L
58,full cream milk,woolworths,2.061190,1.875,100G


In [104]:
idx = counts.groupby(['category'])['Volume'].transform(max) == counts['Volume']
counts[idx]

,category,unit_quantity,Volume,max
0,banana,1EA,5,5
1,eggs,100G,22,22
5,full cream milk,1L,38,38


In [105]:
# Get data
df = pd.DataFrame({'category': ['banana', 'eggs', 'eggs', 'full cream milk', 'full cream milk', 'full cream milk'], 
                    'unit_quantity': ['1EA', '100G', '100ML', '100G', '100ML', '1L'], 
                    'Count': [5, 22, 1, 5, 1, 38],}, 
                   index = [0, 1, 2, 3, 4, 5]) 
df

,category,unit_quantity,Count
0,banana,1EA,5
1,eggs,100G,22
2,eggs,100ML,1
3,full cream milk,100G,5
4,full cream milk,100ML,1
5,full cream milk,1L,38


In [115]:
# Get index of the original for which `Count` is higher
idx = df.groupby(['category'])['Count'].transform(max) == df['Count']

# Get most common `unit_quantity`
df.loc[idx, :]

,category,unit_quantity,Count
0,banana,1EA,5
1,eggs,100G,22
5,full cream milk,1L,38


In [113]:
df.groupby(['category'])['Count'].transform(max)

0     5
1    22
2    22
3    38
4    38
5    38
Name: Count, dtype: int64